**Melhorias**

- Mostrar estrutura da rede em um grafo

https://github.com/amir7d0/classification-neural-network/blob/main/ANN.ipynb

# ES

In [1]:
import numpy as np

def mutation(filho, a):
    for i in range(len(filho)):
        filho[i] += np.random.normal(0, a, size=filho[i].shape)
    return filho

def marriage(pais,p):
    """
    Escolhe os pais para o casamento
    
    Params:
        - pais: população de pais
        - p: numero de pais escolhidos para o casamento
    return:
        - pais_escolhidos: lista de pais escolhidos
        - indices_pais: indices dos pais escolhidos
    """
    # indice dos pais
    pais_ids = np.arange(0,len(pais))
    # escolha p idices aleatorioamente
    indices_pais = np.random.choice(pais_ids,size=p, replace=False)
    # seleciona os p pais referentes aos indices
    pais_escolhidos = [pais[i] for i in indices_pais]

    return pais_escolhidos

def recombination(pais):
    filho = [np.zeros_like(p) for p in pais[0]]
    n_layers = len(pais[0])
    for l in range(n_layers):
        # escolhe o pai
        j = np.random.randint(0, len(pais))
        pai = pais[j][l] 
        # se for a ultima camada       
        if (l==n_layers-1):
            # percorre vetores de pesos da ultima camada
            for c in range(len(filho[l])):
                # percorre valores, exceto bias
                for ci in range(len(filho[l][c])-1):
                    a = pai[c][:-1]
                    filho[l][c][ci] = np.random.choice(a)
                filho[l][c][-1] = pai[c][-1]
        else:
            # print ("Atualizando camada ", l)
            for i in range(filho[l].shape[1]-1):
                # print ("K máximo ", filho[l].shape[1])
                # a ultima coluna é o bias
                k = np.random.randint(0, filho[l].shape[1]-1)
                # print ("coluna k")
                filho[l][:, i] = pai[:, k]
            filho[l][:, -1] = pai[:, -1]
    return filho

def initialize_population(layers, n, init='uniform'):
    population = []
    for i in range(n):
        network = []
        for layer in layers:
            neurons, inputs =  layer.shpW
            if init=='uniform':
                W = np.random.uniform(size=neurons*inputs).reshape(neurons, inputs)
            else:
                W = np.random.randn(neurons, inputs)
            b = np.zeros((neurons, 1))
            p = np.concatenate([W,b], axis=1)
            network.append(p)
        population.append(network)
    return population

# Training

In [27]:
import neural_network as nn
import importlib
import functions

<module 'functions' from '/home/iran/Documentos/code/natural_computing/tarefa_computacional2/functions.py'>

In [80]:
def train_nn(x_train=None, x_vali=None, y_train=None, y_vali=None, epochs=50, layers=[], mu=20, lam=60, p=2, a=0.15, lograte=10, loss='bce', encoder=None):
    """
    Implementa ES(mu+lam)
    Params:
        - x_train

    Returns
        - best: best candidate (Neural Network weights)
        - metrics: [scores, scores_vali, acc_training, acc_validation]
    """
    # para plotagem
    scores = [] 
    scores_vali = []
    acc_training = []
    acc_validation = []
    best, best_acc, best_score = None, 0, np.inf
    # loss e acuracia da validação
    best_score_v = best_score
    best_acc_v = best_acc
    population = initialize_population(layers, lam)
    y_true = encoder.inverse_transform(y_train).flatten()
    y_true_vali = encoder.inverse_transform(y_vali).flatten()

    scores_children = list()
    n_flag = int(epochs/2)
    last_score = -1
    cnt_score = 0
    for epoch in range(epochs):        
        scores_children = list()
        for parent in population:
            s, ac = functions.eval_individual(parent, layers, x_train, y_train, y_true, loss=loss, encoder=encoder)
            s_v, ac_v = functions.eval_individual(parent, layers, x_vali, y_vali, y_true_vali, loss=loss, encoder=encoder)
            if (s < best_score):
                best_acc = ac
                best_acc_v = ac_v

                best_score = s
                best_score_v = s_v
                best = parent
            scores_children.append(s)        
        ranks = np.argsort(np.argsort(scores_children))
        # seleciona os mu pais
        children  = [population[i] for i in ranks[:mu]]
        #scores_children = list()
        for l in range(lam):
            pais_marriage = marriage(population, p)
            filho = recombination(pais_marriage)
            filho = mutation(filho, a)
            children.append(filho)
        
        if (len(scores)>=1 and best_score == scores[-1]):
            cnt_score += 1
        else:
            cnt_score = 0
        if (cnt_score==n_flag): 
            print ("Early stopping (network stop improving)")
            break

        scores.append(best_score)
        scores_vali.append(best_score_v)
        acc_training.append(best_acc)
        acc_validation.append(best_acc_v)

        
        if (lograte > 0) and (epoch%lograte==0):
            print ("#:{} acc_train={:.4f} acc_val={:.4f} loss_train={:.4f} loss_val={:.4f} cnt_score {}".format(epoch, best_acc, best_acc_v, best_score, best_score_v, cnt_score))
        population = children
    return best, [scores, scores_vali, acc_training, acc_validation]

# Multicategorical

In [4]:
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder

## Iris dataset

In [5]:
X, Y = functions.load_dataset('iris')

In [24]:
# x_train, y_train, epochs, mu, lam, p, a, lograte, enc
args_iris = [X_train, X_test, y_hot_train, y_hot_test, 20, 100, 150, 2, 0.2, enc_train]

In [11]:
# definição da estrutura da rede
layers = [nn.Layer(4, 5, 'relu'), nn.Layer(5, 3, 'softmax')]

In [12]:
best, metrics = train_nn(x_train=X_train, x_vali=X_test, y_train=y_hot_train, y_vali=y_hot_test, epochs=500, 
                layers=layers, mu=100, lam=150, p=2, a=0.2, loss='cce', lograte=50,
                encoder=enc_train)

#:0 acc_train=0.3481 acc_val=0.2000 loss_train=0.9771 loss_val=1.0317
#:50 acc_train=0.8370 acc_val=0.8000 loss_train=0.7285 loss_val=0.7424
#:100 acc_train=0.6667 acc_val=0.5333 loss_train=0.6791 loss_val=0.7691
#:150 acc_train=0.8296 acc_val=0.9333 loss_train=0.5767 loss_val=0.4944
#:200 acc_train=0.8296 acc_val=0.9333 loss_train=0.5767 loss_val=0.4944
Early stopping (network stop improving)


In [28]:
functions.run_es_experiments(train_nn, 4, layers, "es_nn_iris", args=args_iris)

Running exp 0
Early stopping (network stop improving)
10
Running exp 1
Early stopping (network stop improving)
10
Running exp 2
Early stopping (network stop improving)
10
Running exp 3
Early stopping (network stop improving)
10


In [39]:
# TODO: aumentar tamanho da população ou da rede
for l in [10, 15, 20]:
    print ("l: ", l)
    layers = [nn.Layer(4, l, 'relu'), nn.Layer(l, 3, 'softmax')]
    a_ = [0.1, 0.2, 0.3, 0.4]
    for a in a_:
        print ("\ta ", a, end=" ")
        args_iris[6] = a
        functions.run_es_experiments(train_nn2, 4, layers, "es_nn_iris", args=args_iris)

a  0.1 loss 0.8158 accuracy 0.5900
a  0.2 loss 0.6788 accuracy 0.6550
a  0.3 loss 0.6316 accuracy 0.7133
a  0.4 loss 0.5521 accuracy 0.7550


In [73]:
layers = [nn.Layer(4, 10, 'relu'), nn.Layer(10, 3, 'softmax')]

## Wine dataset

In [30]:
X, Y = functions.load_dataset('wine')

In [57]:
Y

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 3, 3,
       3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
       3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
       3, 3])

In [62]:
X_train, y_hot_train, X_test, y_hot_test, encoder = get_samples(X, Y, test_size=0.15)

In [59]:
X_train.shape[0], len(np.unique(y_hot_train))

(13, 2)

In [94]:
layers = [nn.Layer(13, 10, 'relu'), nn.Layer(10, 3, 'softmax')]
args_wine = [X_train, X_test, y_hot_train, y_hot_test, 500, 100, 150, 2, 0.2, encoder]

In [98]:
best, metrics = train_nn(x_train=X_train, x_vali=X_test, y_train=y_hot_train, y_vali=y_hot_test, epochs=500, 
                layers=layers, mu=60, lam=110, p=2, a=0.25, loss='cce', lograte=50,
                encoder=encoder)

#:0 acc_train=0.4238 acc_val=0.2593 loss_train=1.1012 loss_val=1.3620 cnt_score 0
#:50 acc_train=0.5894 acc_val=0.4444 loss_train=0.9492 loss_val=0.9352 cnt_score 44
#:100 acc_train=0.5894 acc_val=0.4444 loss_train=0.9492 loss_val=0.9352 cnt_score 94
#:150 acc_train=0.5894 acc_val=0.4444 loss_train=0.9492 loss_val=0.9352 cnt_score 144
#:200 acc_train=0.5894 acc_val=0.4444 loss_train=0.9492 loss_val=0.9352 cnt_score 194
#:250 acc_train=0.5894 acc_val=0.4444 loss_train=0.9492 loss_val=0.9352 cnt_score 244
Early stopping (network stop improving)


In [100]:
layers_size = [2,4,6,8]
for l in layers_size:
    layers = [nn.Layer(13, l, 'relu'), nn.Layer(l, 3, 'softmax')]
    for a in [0.15, 0.25, 0.35, 0.45]:
        print ("l {} a {}".format(l, a))
        best, metrics = train_nn(x_train=X_train, x_vali=X_test, y_train=y_hot_train, y_vali=y_hot_test, epochs=500, 
                    layers=layers, mu=100, lam=150, p=2, a=a, loss='cce', lograte=100,
                    encoder=encoder)
        print ("Loss {:.4f}".format(metrics[0][-1]))

l 2 a 0.15
#:0 acc_train=0.4702 acc_val=0.3704 loss_train=1.0588 loss_val=1.1988 cnt_score 0
#:100 acc_train=0.5033 acc_val=0.3333 loss_train=0.8829 loss_val=0.9724 cnt_score 21
#:200 acc_train=0.6755 acc_val=0.5185 loss_train=0.8729 loss_val=0.9639 cnt_score 65
#:300 acc_train=0.7086 acc_val=0.5926 loss_train=0.7832 loss_val=0.7963 cnt_score 33
#:400 acc_train=0.7086 acc_val=0.5926 loss_train=0.7832 loss_val=0.7963 cnt_score 133
Loss 0.7832
l 2 a 0.25
#:0 acc_train=0.3311 acc_val=0.3333 loss_train=1.0570 loss_val=1.1073 cnt_score 0
#:100 acc_train=0.5762 acc_val=0.4444 loss_train=0.7781 loss_val=1.0097 cnt_score 47
#:200 acc_train=0.6689 acc_val=0.6667 loss_train=0.6628 loss_val=0.6578 cnt_score 29
#:300 acc_train=0.6821 acc_val=0.5926 loss_train=0.6623 loss_val=0.6230 cnt_score 99
#:400 acc_train=0.6821 acc_val=0.5926 loss_train=0.6623 loss_val=0.6230 cnt_score 199
Loss 0.6300
l 2 a 0.35
#:0 acc_train=0.3311 acc_val=0.3333 loss_train=1.0476 loss_val=1.1215 cnt_score 0
#:100 acc_train

KeyboardInterrupt: 

In [87]:
best, metrics = train_nn(x_train=X_train, x_vali=X_test, y_train=y_hot_train, y_vali=y_hot_test, epochs=500, 
                layers=layers, mu=60, lam=110, p=2, a=0.25, loss='cce', lograte=50,
                encoder=encoder)

#:0 acc_train=0.3311 acc_val=0.3333 loss_train=1.0377 loss_val=1.0750 cnt_score 0
#:50 acc_train=0.6623 acc_val=0.5556 loss_train=0.8526 loss_val=1.4622 cnt_score 19
#:100 acc_train=0.6623 acc_val=0.5556 loss_train=0.8526 loss_val=1.4622 cnt_score 69
#:150 acc_train=0.6954 acc_val=0.5556 loss_train=0.6730 loss_val=1.0988 cnt_score 43
#:200 acc_train=0.6954 acc_val=0.5556 loss_train=0.6730 loss_val=1.0988 cnt_score 93
#:250 acc_train=0.6954 acc_val=0.5556 loss_train=0.6730 loss_val=1.0988 cnt_score 143
#:300 acc_train=0.6954 acc_val=0.5556 loss_train=0.6730 loss_val=1.0988 cnt_score 193
#:350 acc_train=0.6954 acc_val=0.5556 loss_train=0.6730 loss_val=1.0988 cnt_score 243
Early stopping (network stop improving)
